In [1]:
# It imports necessary libraries such as pandas for data manipulation, matplotlib for plotting, and tqdm for progress bars.

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import math
from tqdm import tqdm
pd.options.display.float_format = '{:5f}'.format
pd.set_option('display.max_columns', 40)

%matplotlib inline

df = pd.read_csv('/aihub/data/raw/df_bts_BMGF_seoul_vac_cli_F.csv')
df.head(3)

Duplicate key in file PosixPath('/usr/local/lib/python3.8/dist-packages/matplotlib/mpl-data/matplotlibrc'), line 770 ('font.family\t: NanumGothicCoding')
/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (8,9,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,start_time,end_time,latitude,longitude,pcr_result,pcr_date,date_diff,dose_1_date,dose_2_date,dose_3_date,dose_4_date,vac_date_diff,birthday,address,gender
0,3819.000000,2022-03-03 10:49:28,2022-03-03 10:49:51,37.484561,127.033997,1.000000,2022-03-04,1 days,2021-09-17,2021-10-22,NaN,NaN,133.000000,1976-05-05,"06800|서울특별시 서초구 헌릉로8길 45(내곡동, 서초포레스타2단지)",1.000000
1,3819.000000,2022-03-03 10:49:51,2022-03-03 10:50:02,37.484561,127.033997,1.000000,2022-03-04,1 days,2021-09-17,2021-10-22,NaN,NaN,133.000000,1976-05-05,"06800|서울특별시 서초구 헌릉로8길 45(내곡동, 서초포레스타2단지)",1.000000
2,3819.000000,2022-02-25 19:24:39,2022-02-25 19:25:11,37.448386,127.054717,1.000000,2022-03-04,7 days,2021-09-17,2021-10-22,NaN,NaN,133.000000,1976-05-05,"06800|서울특별시 서초구 헌릉로8길 45(내곡동, 서초포레스타2단지)",1.000000


In [2]:
# Data type processing
df['id'] = df['id'].astype(str)
df['dose_1_date'] = pd.to_datetime(df['dose_1_date'])
df['dose_2_date'] = pd.to_datetime(df['dose_2_date'])
df['dose_3_date'] = pd.to_datetime(df['dose_3_date'])
df['dose_4_date'] = pd.to_datetime(df['dose_4_date'])
df['pcr_date'] = pd.to_datetime(df['pcr_date'])
df['start_time'] = pd.to_datetime(df['start_time'])
df['end_time'] = pd.to_datetime(df['end_time'])
df['pcr_result'] = pd.Categorical(df['pcr_result'], categories=[2, 1])

In [3]:
df.head(3)

,id,start_time,end_time,latitude,longitude,pcr_result,pcr_date,date_diff,dose_1_date,dose_2_date,dose_3_date,dose_4_date,vac_date_diff,birthday,address,gender,last_vac_date
0,3819.0,2022-03-03 10:49:28,2022-03-03 10:49:51,37.484561,127.033997,1,2022-03-04,1 days,2021-09-17,2021-10-22,NaT,NaT,133.000000,1976-05-05,"06800|서울특별시 서초구 헌릉로8길 45(내곡동, 서초포레스타2단지)",1.000000,2021-10-22
1,3819.0,2022-03-03 10:49:51,2022-03-03 10:50:02,37.484561,127.033997,1,2022-03-04,1 days,2021-09-17,2021-10-22,NaT,NaT,133.000000,1976-05-05,"06800|서울특별시 서초구 헌릉로8길 45(내곡동, 서초포레스타2단지)",1.000000,2021-10-22
2,3819.0,2022-02-25 19:24:39,2022-02-25 19:25:11,37.448386,127.054717,1,2022-03-04,7 days,2021-09-17,2021-10-22,NaT,NaT,133.000000,1976-05-05,"06800|서울특별시 서초구 헌릉로8길 45(내곡동, 서초포레스타2단지)",1.000000,2021-10-22


In [4]:
# Create a date-related variable
df['stay_time'] = df['end_time'] - df['start_time']
df['month'] = df['start_time'].dt.month
df['hour'] = df['start_time'].dt.hour
df['weekday_cri'] = [4 if x.dayofweek < 5 else 5 for x in df['start_time']]

In [5]:
# Filtering 1
# Delete an ID with less than 7 days of past activity
tmp_date_diff_cnt = df.groupby(['id', 'pcr_date'], as_index=False)['date_diff'].nunique()
id_not_7date_diff = tmp_date_diff_cnt[tmp_date_diff_cnt['date_diff'] != 7]['id']
df_filtered = df[~df['id'].isin(id_not_7date_diff)]
print(df_filtered['id'].nunique())
# Extract only data with the last PCR_DATE by ID
def last_pcr(df):
    max_pcr_date = df['pcr_date'].max()
    return df[df['pcr_date'] == max_pcr_date]

print(df_filtered.shape)
print(df_filtered['pcr_date'].nunique())
df_filtered = df_filtered.groupby('id', as_index=False).apply(lambda x: last_pcr(x))
print(df_filtered.shape)
print('id 별 pcr_date 개수', df_filtered.groupby('id', as_index=False)['pcr_date'].nunique()['pcr_date'].sum())

837
(3732591, 21)
125
(3226937, 21)
id 별 pcr_date 개수 837


In [6]:
# Create a location ID
df_filtered['loc_id'] = [f'{str(x)[:9]}_{str(y)[:10]}' for x, y in zip(df_filtered['latitude'], df_filtered['longitude'])]

In [7]:
# Residence classification
## Extracting nighttime residences
def get_residence_loc(df):
    cond = (df['hour'] >= 22) | (df['hour'] < 7)
    counts = df[cond]['loc_id'].value_counts()

    if len(counts) < 1:
        return None
    
    return counts.idxmax()

df_id_resi = df_filtered.groupby('id', as_index=False).apply(lambda x: get_residence_loc(x))
df_id_resi.columns = ['id', 'loc_id']
df_id_resi['loc_type'] = 'resi'

In [8]:
df_merged_resi = pd.merge(df_filtered, df_id_resi, how='left', on=['id', 'loc_id'])
df_merged_resi['loc_type'].fillna('outside', inplace=True)
df_merged_resi.tail(3)

,id,start_time,end_time,latitude,longitude,pcr_result,pcr_date,date_diff,dose_1_date,dose_2_date,dose_3_date,dose_4_date,vac_date_diff,birthday,address,gender,last_vac_date,stay_time,month,hour,weekday_cri,loc_id,loc_type
3226934,64746.0,2022-06-03 16:13:37,2022-06-03 16:13:49,37.576528,126.907777,2,2022-06-07,4 days,NaT,NaT,NaT,NaT,NaN,1985-09-01,서울특별시 서대문구 북가좌동,1.000000,NaT,0 days 00:00:12,6,16,4,37.576528_126.907777,outside
3226935,64746.0,2022-06-03 15:30:36,2022-06-03 15:30:47,37.576528,126.907777,2,2022-06-07,4 days,NaT,NaT,NaT,NaT,NaN,1985-09-01,서울특별시 서대문구 북가좌동,1.000000,NaT,0 days 00:00:11,6,15,4,37.576528_126.907777,outside
3226936,64746.0,2022-06-03 16:16:33,2022-06-03 16:16:43,37.576528,126.907777,2,2022-06-07,4 days,NaT,NaT,NaT,NaT,NaN,1985-09-01,서울특별시 서대문구 북가좌동,1.000000,NaT,0 days 00:00:10,6,16,4,37.576528_126.907777,outside


In [9]:
# Extract only outside
df_outside = df_merged_resi[df_merged_resi['loc_type'] == 'outside']
print(df_outside['id'].nunique())
tmp_outside_cnt = df_outside.groupby('id', as_index=False).apply(lambda x: x.shape[0])
tmp_outside_cnt.columns = ['id', 'cnt']
tmp_outside_cnt[tmp_outside_cnt['cnt'] < 1]

837


,id,cnt


In [10]:
df_outside.head(3)

,id,start_time,end_time,latitude,longitude,pcr_result,pcr_date,date_diff,dose_1_date,dose_2_date,dose_3_date,dose_4_date,vac_date_diff,birthday,address,gender,last_vac_date,stay_time,month,hour,weekday_cri,loc_id,loc_type
0,20289.0,2022-03-24 00:59:07,2022-03-24 00:59:07,37.508128,127.021161,2,2022-03-31,7 days,2021-08-20,NaT,2021-11-30,NaT,121.000000,1996-05-04,서울특별시 서초구 반포1동,1.000000,2021-11-30,0 days 00:00:00,3,0,4,37.508127_127.021161,outside
1,20289.0,2022-03-27 07:50:24,2022-03-27 07:51:19,37.572034,127.079539,2,2022-03-31,4 days,2021-08-20,NaT,2021-11-30,NaT,121.000000,1996-05-04,서울특별시 서초구 반포1동,1.000000,2021-11-30,0 days 00:00:55,3,7,5,37.572033_127.079538,outside
2,20289.0,2022-03-27 19:33:42,2022-03-27 19:36:51,37.507015,127.034314,2,2022-03-31,4 days,2021-08-20,NaT,2021-11-30,NaT,121.000000,1996-05-04,서울특별시 서초구 반포1동,1.000000,2021-11-30,0 days 00:03:09,3,19,5,37.507014_127.034313,outside


In [11]:
from datetime import timedelta

df_outside['cal_date_diff'] = df_outside['date_diff'].str.split(' ').str[0].astype('int')
df_outside['today'] = df_outside.apply(lambda x : x['pcr_date'] - timedelta(days = x['cal_date_diff']), axis = 1)
df_outside['end_hour'] = df_outside['end_time'].dt.hour

<ipython-input-11-c2d90a6cc803>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_outside['cal_date_diff'] = df_outside['date_diff'].str.split(' ').str[0].astype('int')
<ipython-input-11-c2d90a6cc803>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_outside['today'] = df_outside.apply(lambda x : x['pcr_date'] - timedelta(days = x['cal_date_diff']), axis = 1)
<ipython-input-11-c2d90a6cc803>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [12]:
# check counts per test result
df_outside['pcr_result'].value_counts()

2    1703810
1     298240
Name: pcr_result, dtype: int64

# Position Overlap Calculation Code

In [13]:
# multiprocessing calculation code

from multiprocessing import Pool

# Split the apply operation of GROUP BY to PROCESS in parallel
def applyParallel(dfGrouped, func):
    with Pool(45) as p: # Set the number of CPU cores to -1
        result = p.map(func, [group for _, group in dfGrouped])
    return pd.concat(result)

In [25]:
# Function for calculating for overlap 
def check_overlap(df):
    df.reset_index(drop=True, inplace=True)

    # Check for different IDs (people) and overlapping times
    for i, ref_row in df.iterrows():
        for _, target_row in df.iterrows():
            if ref_row['id'] == target_row['id']:
                continue

            cond1 = (ref_row['start_time'] >= target_row['start_time']) and (ref_row['start_time'] <= target_row['end_time'])
            cond2 = (ref_row['end_time'] >= target_row['start_time']) and (ref_row['end_time'] <= target_row['end_time'])
            cond3 = (target_row['start_time'] > ref_row['start_time']) and (target_row['end_time'] < ref_row['end_time'])
            
            if cond1 or cond2 or cond3:
                #print(ref_row['loc_id'])
                overlap_start = max(ref_row['start_time'], target_row['start_time'])
                overlap_end = min(ref_row['end_time'], target_row['end_time'])
                time_diff = overlap_end - overlap_start
                df.loc[i, 'overlaped_seconds'].append(time_diff.seconds) # Add overlapping times to the list
                df.loc[i, 'overlaped_people'].append(target_row['id']) # Add overlapping IDs to the list
                df.loc[i, 'overlap_cnt'] += 1
    
    return df

In [ ]:
# compute overlap of all loc_type-independent data
df_overlap_tmp = df_merged_resi.copy()
df_overlap_tmp['id'] = df_overlap_tmp['id'].astype('float').astype('int').astype('str') # clean up id type
df_overlap_tmp['overlap_cnt'] = 0
df_overlap_tmp['overlaped_seconds'] = [[] for _ in range(df_overlap_tmp.shape[0])]
df_overlap_tmp['overlaped_people'] = [[] for _ in range(df_overlap_tmp.shape[0])]
df_overlap = applyParallel(df_overlap_tmp.groupby('loc_id', as_index=False), check_overlap)

In [28]:
df_overlap.head()

,id,start_time,end_time,latitude,longitude,pcr_result,pcr_date,date_diff,dose_1_date,dose_2_date,dose_3_date,dose_4_date,vac_date_diff,birthday,address,gender,last_vac_date,stay_time,month,hour,weekday_cri,loc_id,loc_type,overlap_cnt,overlaped_seconds,overlaped_people
0,46104,2022-03-22 12:37:58,2022-03-22 12:38:34,33.116939,126.268908,2,2022-03-24,2 days,NaT,NaT,2022-01-27,NaT,56.000000,1996-09-03,서울특별시 종로구 명륜1가,1.000000,NaT,0 days 00:00:36,3,12,4,33.116938_126.268908,outside,0,[],[]
0,34827,2022-02-24 10:47:23,2022-02-24 10:47:50,33.200194,126.270711,2,2022-03-02,6 days,2021-08-30,2021-10-05,NaT,NaT,148.000000,1995-11-08,서울특별시 구로구 가리봉동,1.000000,2021-10-05,0 days 00:00:27,2,10,4,33.200194_126.270711,outside,0,[],[]
1,60173,2022-04-10 18:57:10,2022-04-10 18:57:18,33.200194,126.270711,2,2022-04-12,2 days,2021-08-18,2021-09-17,2022-02-07,NaT,64.000000,1995-04-13,서울특별시 중구 황학동,1.000000,2022-02-07,0 days 00:00:08,4,18,5,33.200194_126.270711,outside,0,[],[]
2,60173,2022-04-10 18:55:05,2022-04-10 18:55:06,33.200194,126.270711,2,2022-04-12,2 days,2021-08-18,2021-09-17,2022-02-07,NaT,64.000000,1995-04-13,서울특별시 중구 황학동,1.000000,2022-02-07,0 days 00:00:01,4,18,5,33.200194_126.270711,outside,0,[],[]
3,60173,2022-04-10 17:30:47,2022-04-10 17:30:48,33.200194,126.270711,2,2022-04-12,2 days,2021-08-18,2021-09-17,2022-02-07,NaT,64.000000,1995-04-13,서울특별시 중구 황학동,1.000000,2022-02-07,0 days 00:00:01,4,17,5,33.200194_126.270711,outside,0,[],[]


In [33]:
df_overlap.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3226937 entries, 0 to 17
Data columns (total 26 columns):
 #   Column             Dtype          
---  ------             -----          
 0   id                 object         
 1   start_time         datetime64[ns] 
 2   end_time           datetime64[ns] 
 3   latitude           float64        
 4   longitude          float64        
 5   pcr_result         category       
 6   pcr_date           datetime64[ns] 
 7   date_diff          object         
 8   dose_1_date        datetime64[ns] 
 9   dose_2_date        datetime64[ns] 
 10  dose_3_date        datetime64[ns] 
 11  dose_4_date        datetime64[ns] 
 12  vac_date_diff      float64        
 13  birthday           object         
 14  address            object         
 15  gender             float64        
 16  last_vac_date      datetime64[ns] 
 17  stay_time          timedelta64[ns]
 18  month              int64          
 19  hour               int64          
 20  weekday

In [ ]:
# Save the dataframe
df_overlap.reset_index(drop=True, inplace=True)
df_overlap.to_pickle('/aihub/data/raw/df_overlap_with_people.pkl')